# Hello!
This is a walkthrough to train the University of Waterloo's submission model at ClimateHack 2023.

To find all our experiments and code, see our original [repo](https://github.com/trevor-yu-087/climatehack.ai-2024), but beware, it is not documented, or well organized.

# Environment set up

We use docker to package dependencies. If you are using VScode or a JetBrains IDE, the devcontainers extension should be able to use the .devcontainer directory to build the docker image and use it as a development environment.

If you do not want to use docker, you can (hopefully) get set up by running:

- `pip install -r local-requirements.txt`
- `conda install cartopy`
- `pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121`

A machine with a CUDA enabled GPU is required.

In [ ]:
import huggingface_hub
from os import makedirs, path
import pandas as pd
import numpy as np
import pickle
from dataset import get_datasets
import yaml
import torch
from pvlib.solarposition import get_solarposition
from datetime import datetime
from cartopy import crs
from itertools import accumulate
from functools import partial
from perceiver import PVPerceiver
import pytorch_lightning as L
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

# Download data
Our model used pv, hrv and weather data.

For this example we'll only be downloading a few months of data.

Note that you have to download the [indices.json](https://github.com/climatehackai/getting-started-2023/blob/main/indices.json) file and place it in the same directory as the data that gets downloaded below.

In [ ]:
datadir = "/workspaces/waterloo-climatehack/data" # change this
makedirs(datadir, exist_ok=True)

huggingface_hub.snapshot_download(
    repo_id="climatehackai/climatehackai-2023", 
    local_dir=datadir, 
    cache_dir=datadir + '/cache',
    local_dir_use_symlinks=False, 
    repo_type="dataset",
    ignore_patterns=["aerosols/*", "satellite-nonhrv/*"],
    allow_patterns=["*10.zarr.zip", "*11.zarr.zip", "*.parquet", "*metadata.csv"]
)

# Generating PV  Features
We generate site specific features (such as the site's max and average output during each month).

In [ ]:
years = [2020, 2021]
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

for i, month_name in enumerate(months):
    print(f'Processing {month_name}')
    month = pd.read_parquet([datadir + f'/pv/{year}/{i+1}.parquet' for year in [2020,2021]])
    month = month.drop(['generation_wh'], axis=1).reorder_levels(['ss_id', 'timestamp'])

    site_ids = month.index.get_level_values(0).unique().values

    monthly_avg, monthly_max, monthly_average_max = [], [], []

    for site in site_ids:
        a = month.loc[site].between_time('5:00', '22:00')
        monthly_max.append(a.power.max())
        monthly_avg.append(a.power.mean())
        monthly_average_max.append(a.groupby([a.index.hour, a.index.minute]).power.mean().max())

    frame = pd.DataFrame(np.array([monthly_avg, monthly_max, monthly_average_max]).T, index=site_ids)
    frame.columns = [f'{month_name}_avg', f'{month_name}_max', f'{month_name}_average_max']

    if i == 0:
        pv_metrics_frame = frame
    else:
        pv_metrics_frame = pv_metrics_frame.join(frame)

In [ ]:
pv_metrics_frame.head()

### Loading PV Metadata

In [ ]:
metadata = pd.read_csv(datadir + '/pv/metadata.csv')
metadata.index = metadata.ss_id
metadata.drop(['llsoacd', 'operational_at', 'ss_id'], axis=1, inplace=True)
metadata.head()

### Converting the PV Metrics Dataframe to a Mapping

In [ ]:
pv_metric_sites = set(pv_metrics_frame.index)
nan_fill = pv_metrics_frame.mean()

In [ ]:
pv_metrics = {}

month_names = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]

for site_id, (lat, lon, orient, tilt, kwp) in metadata[["latitude_rounded", "longitude_rounded", "orientation", "tilt", "kwp"]].iterrows():
    for month_number, month in enumerate(month_names, start=1):
        key = (lat, lon, orient, tilt, kwp)
        metric_names = ["_".join([month, metric]) for metric in ["avg", "max", "average_max"]]
        if site_id not in pv_metric_sites:
            metrics = nan_fill[metric_names].values
        else:
            metrics = pv_metrics_frame.loc[site_id, metric_names].values
        if np.isnan(metrics).any():
            metrics = nan_fill[metric_names].values
        pv_metrics.setdefault(month_number, {})[key] = metrics

In [ ]:
with open(datadir + "/pv_metrics.pkl", "wb") as f:
    pickle.dump(pv_metrics, f)

# Data Loading
Now let's run our dataset class to validate that our data is set up correctly.

In [ ]:
# yaml file that is used to configure training runs
CONFIG_FILE_NAME = "train.yaml"

with open(CONFIG_FILE_NAME) as file:
    config = yaml.safe_load(file)

In [ ]:
train_ds, test_ds = get_datasets(
    config["data_path"],
    (config["start_date"], config["end_date"]),
    batch_size=config["batch_size"],
    hrv="hrv" in config["modalities"],
    weather="weather" in config["modalities"],
    metadata="metadata" in config["modalities"],
    seed=config["seed"],
    pv_features_file=config["pv_features_file"],
    test_size=config["test_size"],
    hrv_crop=config["hrv_crop"],
    weather_crop=config["weather_crop"],
    zipped=config["zipped"],
    offset_start_time=config["offset_start_time"]
)

In [ ]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.manual_seed(config["seed"])
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Precompile solar position
_ = get_solarposition(
    time=datetime(2020, 1, 2, 3),
    latitude=123.4,
    longitude=49.0,
    method="nrel_numba"
)

In [ ]:
# hrv lat lon features
delta_geos = 1000.1343488693237
ix = np.arange(config["hrv_crop"]) - (config["hrv_crop"] // 2)
xx, yy = np.meshgrid(ix, ix)
xx_hrv = xx * delta_geos
yy_hrv = yy * delta_geos


# weather lat lon features
delta_nwp = 0.0623
ix = np.arange(config["weather_crop"]) - (config["weather_crop"] // 2)
xx, yy = np.meshgrid(ix, ix)
xx_nwp = xx * delta_nwp
yy_nwp = yy * delta_nwp

In [ ]:
# convert between geostationary and lat lon coord systems
hrv_coords = crs.Geostationary(central_longitude=9.5, sweep_axis="y")
latlon_coords = crs.Geodetic()   
def get_hrv_lat_lon_features(lat, lon):
    x_geo, y_geo = hrv_coords.transform_point(lon, lat, latlon_coords)

    xx_geo = xx_hrv + x_geo
    yy_geo = yy_hrv + y_geo

    coords = latlon_coords.transform_points(
        hrv_coords,
        xx_geo, yy_geo
    )

    xx_lon = coords[..., 0]
    yy_lat = coords[..., 1]
    features = np.stack((xx_lon, yy_lat), axis=-1)
    return features

In [ ]:
def compute_solar_incidence(az, el, orient, tilt):
    # Assume in degrees
    panel_vec = np.array([
        np.cos(np.radians(orient)),
        np.sin(np.radians(orient)),
        np.sin(np.radians(tilt))
    ])

    solar_vec = np.stack([
        np.cos(np.radians(az)),
        np.sin(np.radians(az)),
        np.sin(np.radians(el))
    ], axis=1)

    sim = -solar_vec @ panel_vec.T
    return sim

In [ ]:
def worker_init_fn(id, split_seed: int):
    process_seed = torch.initial_seed()
    base_seed = process_seed - id
    ss = np.random.SeedSequence(
        [id, base_seed, split_seed]
    )
    np_rng_seed = ss.generate_state(4)
    np.random.seed(np_rng_seed)

In [ ]:
def metadata_collate_fn(batch):
    """Data is already batched
    Weather is already shape (B, C, L, H, W)
    """
    batch = batch[0]
    metadata_features = {}
    for (lat, lon, orient, tilt, kwp), t0 in zip(batch["metadata"], batch["time"]):
        t0 = pd.Timestamp(t0) - pd.Timedelta(hours=1)
        # 60 timestamp including first hour and prediction window
        ts = list(accumulate([pd.Timedelta(minutes=5)] * 59, initial=t0))
        ts = pd.DatetimeIndex(ts)
        solar_pos = get_solarposition(
            time=ts, 
            latitude=lat,
            longitude=lon,
            method="nrel_numba"
        )

        # Scale to [0, 1] for SSP
        doy = ts.day_of_year.values / 365
        mod = ((ts.hour.values * 60) + ts.minute.values) / (24 * 60)
        
        metadata_features.setdefault("time", []).append(np.stack([
            mod,
            doy
        ], axis=1))

        # Weather time features on the hour
        t0 = t0.floor("60min")  # t0 already 1 hr before
        ts = list(accumulate([pd.Timedelta(minutes=60)] * 5, initial=t0))
        ts = pd.DatetimeIndex(ts)
        doy = ts.day_of_year.values / 365
        mod = ((ts.hour.values * 60) + ts.minute.values) / (24 * 60)
        metadata_features.setdefault("weather_time", []).append(np.stack([
            mod,
            doy
        ], axis=1))

        lon_xx = lon + xx_nwp
        lat_yy = lat + yy_nwp
        metadata_features.setdefault("location", []).append(np.stack([
            lon_xx,
            lat_yy
        ], axis=-1))

        metadata_features.setdefault("hrv_location", []).append(get_hrv_lat_lon_features(lat, lon))

        # Scale to [0, 1] for SSP
        az = solar_pos["azimuth"].values / 360
        el = solar_pos["apparent_elevation"].values / 360
        metadata_features.setdefault("azel", []).append(np.stack([
            az,
            el
        ], axis=1))

        # Scale to [0, 1] for SSP
        orient = orient / 360
        tilt = tilt / 360
        metadata_features.setdefault("static", []).append(np.array([
            [orient,
            tilt,
            kwp]
        ]))
        
    batch = {k: torch.FloatTensor(v) for k, v in batch.items() if k not in ["time", "metadata"]}
    for k, v in metadata_features.items():
        batch[k] = torch.FloatTensor(np.stack(v))
    batch["pv"] = batch["pv"].unsqueeze(-1)
    batch["pv_features"] = batch["pv_features"].unsqueeze(-2)
    return batch

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_ds,
    batch_size=1,
    shuffle=False,
    collate_fn=metadata_collate_fn,
    pin_memory=True,
    worker_init_fn=partial(worker_init_fn, split_seed=0),
    num_workers=config["num_workers"]
    )
val_loader = torch.utils.data.DataLoader(
    test_ds, 
    batch_size=1, 
    shuffle=False, 
    collate_fn=metadata_collate_fn, 
    pin_memory=True,
    worker_init_fn=partial(worker_init_fn, split_seed=0),
    num_workers=config["num_workers"]
)

# Model Training

In [ ]:
total_steps = len(train_loader) * config["epochs"]
model = PVPerceiver(
    total_steps,
    embedding_dim=config["dim"], 
    num_transformer_layer=config["num_layers"],
    nwp_window=config["weather_crop"], 
    lr=config["lr"], 
    wd=config["wd"],
    init_std=config["init_std"],
    num_latents=config["num_latents"],
    bias=config["bias"],
    pv_latent=config["pv_latent"],
    num_pv_features=config["num_pv_features"]
)

In [ ]:
timestr = datetime.now().strftime("%Y-%m-%d")
save_dir = path.join(config["results_path"], f"{timestr}_{config['run_name']}")
makedirs(save_dir, exist_ok=True)

trainer = L.Trainer(
    default_root_dir=save_dir,
    max_epochs=config["epochs"],
    callbacks=[
        ModelCheckpoint(filename=timestr + "{epoch}-{step}-{val_mae:.3f}", monitor="val_mae", mode="min", save_top_k=3),
        LearningRateMonitor(logging_interval='step', log_momentum=True)
    ],
    log_every_n_steps=50
)

In [ ]:
trainer.fit(
    model, 
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)